In [1]:
pip install scikeras[tensorflow] --upgrade --force-reinstall



     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 187.9 kB/s eta 0:00:08
     - -------------------------------------- 0.1/1.5 MB 297.7 kB/s eta 0:00:05
     - -------------------------------------- 0.1/1.5 MB 326.8 kB/s eta 0:00:05
     --- ------------------------------------ 0.1/1.5 MB 409.6 kB/s eta 0:00:04
     --- ------------------------------------ 0.1/1.5 MB 473.7 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.5 MB 513.4 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.5 MB 518.8 kB/s eta 0:00:03
     ------ --------------------------------- 0.2/1.5 MB 514.3 kB/s eta 0:00:03
     ------- -------------------------------- 0.3/1.5 MB 571.2 kB/s eta 0:00:03
     -------- ------------------------------- 0.3/1.5 MB 593.9 kB/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 2.84.0 requires uritemplate<5,>=3.0.1, which is not installed.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
botocore 1.29.76 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.0 which is incompatible.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.
python-lsp-server 1.7.2 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


   -- ------------------------------------ 20.4/272.9 MB 208.6 kB/s eta 0:20:11
   -- ------------------------------------ 20.4/272.9 MB 206.5 kB/s eta 0:20:23
   -- ------------------------------------ 20.4/272.9 MB 206.5 kB/s eta 0:20:23
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.4/272.9 MB 204.6 kB/s eta 0:20:35
   -- ------------------------------------ 20.5/272.9 MB 200.5 kB/s eta 0:21:00
   -- ---------------------------------

In [ ]:
pip install --upgrade adapt


In [ ]:
pip install scikeras[tensorflow]

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier

# Reading the data
data = pd.read_csv("PQM_FACTORS.csv")

# Encoding the target variable
le = LabelEncoder()
data["Seriousness"] = le.fit_transform(data["Seriousness"])

# Separating features and target variable
features = data.columns.difference(['Seriousness'])
X = data[features]
Y = data["Seriousness"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardizing features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creating a simple neural network model
def create_model(optimizer='adam', activation='relu'):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    return model


# Train the model
model=KerasClassifier(model=create_model, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

# Define the hyperparameters to tune
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30]
}

# Use GridSearchCV to find the best combination of hyperparameters
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

'''
# Evaluate the model on the test set
y_pred_nn = model.predict(X_test)
y_pred_nn = (y_pred_nn > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate accuracy
accuracy_nn = accuracy_score(y_test, y_pred_nn)
print("Neural Network Accuracy:", accuracy_nn)
'''

# Display the best hyperparameters and corresponding accuracy
print(f"Best Parameters: {grid_result.best_params_}")
print(f"Best Accuracy: {grid_result.best_score_}")

Best Parameters: {'batch_size': 16, 'epochs': 10, 'optimizer': 'adam'}
Best Accuracy: 0.10166666666666667


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, LSTM, Embedding, Flatten, Dropout

# Reading the data
data = pd.read_csv("PQM_FACTORS.csv")

# Encoding the target variable
le = LabelEncoder()
data["Seriousness"] = le.fit_transform(data["Seriousness"])

# Separating features and target variable
features = data.columns.difference(['Seriousness'])
X = data[features]
Y = data["Seriousness"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Standardizing features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshaping data for Conv1D layer
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Creating a new neural network model
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
accuracy = model.evaluate(X_test, y_test)[1]
print("New Model Accuracy:", accuracy)
